In [1]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}
response = requests.get("https://www.d3football.com/scoreboard/2024/composite?view=1",
                       headers = headers
                       )

print(response.status_code)

200


In [2]:
print(response.text[500:1500])

>
<title>2024 Division III football scoreboard -  D3football</title>





    
    
			
	<link rel="canonical" href="https://www.d3football.com/scoreboard/2024/composite" />
    <meta property="fb:app_id" content="280467664480" />

    
            <meta property="og:locale" content="en_US" />
        <meta property="og:title"  content="2024 Division III football scoreboard"  />
        <meta property="dcterms.title"  content="2024 Division III football scoreboard"  />
        <meta property="og:description"  content="The full schedule for all 240 teams playing Division III football this season, including links to broadcasts, live stats and scores and stats for completed games."  />
        <meta property="dcterms.description"  content="The full schedule for all 240 teams playing Division III football this season, including links to broadcasts, live stats and scores and stats for completed games."  />
                  <meta property="og:image" content="https://www.d3football.com/i


In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')

title = soup.find('title')
print(title.text)

2024 Division III football scoreboard -  D3football


In [4]:
divs = soup.find_all('div')

for i, div in enumerate(divs[:10]):
    class_name = div.get('class', 'no class')
    print (f"Div {i}: {class_name}")

Div 0: no class
Div 1: ['page', 'clearfix']
Div 2: ['wrapper', 'clearfix']
Div 3: ['container', 'clearfix']
Div 4: ['mobile-menu-bar', 'clearfix']
Div 5: ['d3-logos', 'clearfix']
Div 6: ['clearfix']
Div 7: ['social-links']
Div 8: ['offscreen']
Div 9: ['submenu', 'level1', 'clearfix']


In [5]:
game_rows = soup.find_all('tr', class_ = ['roster-row0', 'roster-row1'])

print(f"Found {len(game_rows)} games")

if game_rows:
    first_game = game_rows[0]
    print("\nFirst Game HTML")
    print(first_game.prettify()[:500])

Found 113 games

First Game HTML
<tr class="roster-row1">
 <td nowrap="" valign="top" width="8%">
  Sep. 7
 </td>
 <td valign="top">
  No. 2
  <a href="/teams/Cortland/2024">
   Cortland
  </a>
 </td>
 <td align="center" style="padding-left: 8px; padding-right: 8px;" valign="top">
  <span>
   76
  </span>
 </td>
 <td valign="top">
  <a href="/teams/Hilbert/2024">
   Hilbert
  </a>
 </td>
 <td align="center" style="padding-left: 8px; padding-right: 8px;" valign="top">
  <span>
   0
  </span>
 </td>
 <td valign="top">
  Final
 </


In [6]:
games_data = []

for game in game_rows[:5]:
    cells = game.find_all('td')

    #extract the date
    date = cells[0].text.strip()

    #extract the teams (look for <a> tags)
    team_links = game.find_all('a')
    team1 = team_links[0].text.strip() if len(team_links) > 0 else "Unknown"
    team2 = team_links[1].text.strip() if len(team_links) > 1 else "Unknown"

    #extract scores (look for <span> tags)
    score_spans = game.find_all('span')
    score1 = score_spans[0].text.strip() if len(score_spans) > 0 else "Unknown"
    score2 = score_spans[1].text.strip() if len(score_spans) > 1 else "0"

    # Store the game data
    game_info = {
        'date': date,
        'team1': team1,
        'score1': score1,
        'team2': team2,
        'score2': score2
    }

    games_data.append(game_info)
    print(f"{date}: {team1} {score1} - {team2} {score2}")

Sep. 7: Cortland 76 - Hilbert 0
Sep. 7: Carthage 0 - St. John's 49
Sep. 7: Hardin-Simmons 38 - Howard Payne 6
Sep. 7: Mount Union 55 - Ferrum 6
Sep. 6: Bridgewater 7 - Susquehanna 41


In [7]:
import pandas as pd

#get all games
all_games = []

for game in game_rows:
    cells = game.find_all('td')
    team_links = game.find_all('a')
    score_spans = game.find_all('span')

    game_data = {
        'date' : cells[0].text.strip(),
        'team1' : team_links[0].text.strip() if len(team_links)>0  else "Unknown",
        'score1' : int(score_spans[0].text.strip()) if len(score_spans)>0 else 0,
        'team2' : team_links[1].text.strip() if len(team_links)>1 else "Unknown",
        'score2' : int(score_spans[1].text.strip()) if len(score_spans) > 1 else 0,
    }
    all_games.append(game_data)

df = pd.DataFrame(all_games)
print(f"Scraped {len(df)} games!")
print("\nFirst 10 games: ")
df.head(25)

Scraped 113 games!

First 10 games: 


date                team1  score1                     team2  score2
0   Sep. 7             Cortland      76                   Hilbert       0
1   Sep. 7             Carthage       0                St. John's      49
2   Sep. 7       Hardin-Simmons      38              Howard Payne       6
3   Sep. 7          Mount Union      55                    Ferrum       6
4   Sep. 6          Bridgewater       7               Susquehanna      41
5   Sep. 7             Lakeland       0            UW-Platteville      65
6   Sep. 7   Washington and Lee      26                 Salisbury      48
7   Sep. 7          Rose-Hulman      14                    DePauw      57
8   Sep. 7             Endicott      44              St. Lawrence      13
9   Sep. 7             Wartburg      38                  Monmouth      24
10  Sep. 7               Geneva       0                Grove City      41
11  Sep. 7      Carnegie Mellon      58                     Thiel       0
12  Sep. 7              Denison       7                  Linfield      31
13  Sep. 6               Aurora      33                   Dubuque      21
14  Sep. 7                Loras       0                      Hope      49
15  Sep. 7          St. Vincent       6  Washington and Jefferson      59
16  Sep. 5         UW-La Crosse      35                       RPI       0
17  Sep. 7        Johns Hopkins      27                    Ithaca      23
18  Sep. 7       Wheaton (Ill.)      14                UW-Oshkosh      21
19  Sep. 7            Whitworth      40         Gustavus Adolphus       7
20  Sep. 6  Western New England      32               Springfield      38
21  Sep. 7         John Carroll       7             UW-Whitewater      34
22  Sep. 7              Cornell      10                       Coe      55
23  Sep. 5        Buffalo State       0                 Brockport      27
24  Sep. 5                Rowan      35                 Stevenson      16

In [8]:
highest_scoring_game = df.loc[df['score1'] + df['score2'] == (df['score1'] + df['score2']).max()]
print("Highest scoring game:")
print(highest_scoring_game)

# What's the average total score?
df['total_score'] = df['score1'] + df['score2']
print(f"\nAverage total score per game: {df['total_score'].mean():.1f}")

# Save to CSV for later use
df.to_csv('d3_football_scores_2024.csv', index=False)
print("\nSaved data to 'd3_football_scores_2024.csv'!")

Highest scoring game:
      date       team1  score1    team2  score2
46  Sep. 7  Gettysburg      61  Juniata      41

Average total score per game: 49.1

Saved data to 'd3_football_scores_2024.csv'!


In [9]:
week_num = 16 
url = f"https://www.d3football.com/scoreboard/2024/composite?view={week_num}"

response = requests.get(url, headers = headers)
print(f"Week {week_num} status : {response.status_code}")

soup_week2 = BeautifulSoup(response.text, 'html.parser')
games_week2 = soup_week2.find_all('tr', class_=['roster-row0', 'roster-row1'])
print(f"Week {week_num} has {len(games_week2)} games")

Week 16 status : 200
Week 16 has 2 games


In [10]:
def scrape_week(year_num, week_num):
    """Scrape games from a specific year and week from D3Football.com"""
    url = f"https://www.d3football.com/scoreboard/{year_num}/composite?view={week_num}"

    response = requests.get(url, headers=headers)
    if response.status_code != 200 :
        print(f"Error retrieving week {week_num} in {year_num}")
        return []

    #parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    game_rows = soup.find_all('tr', class_=['roster-row0','roster-row1'])

    #extract games
    week_games = []
    for game in game_rows:
        cells = game.find_all('td')
        team_links = game.find_all('a')
        score_spans = game.find_all('span')


        # Helper function to safely convert to int
        def safe_int(text):
            try:
                return int(text.strip())
            except (ValueError, AttributeError):
                return None  # Return None for missing/invalid scores

        
        game_data = {
            'year': year_num,
            'week': week_num,  # Add week number!
            'date': cells[0].text.strip(),
            'team1': team_links[0].text.strip() if len(team_links) > 0 else "Unknown",
            'score1': safe_int(score_spans[0].text.strip()) if len(score_spans) > 0 else None,
            'team2': team_links[1].text.strip() if len(team_links) > 1 else "Unknown",
            'score2': safe_int(score_spans[1].text.strip()) if len(score_spans) > 1 else None
        }
        week_games.append(game_data)

    return week_games

week1_2024_games = scrape_week(2023,2)

print(f"Week 2: {len(week1_2024_games)} games")

Week 2: 103 games


In [11]:
#scrape multiple seasons

import time

all_games = []
years = [2022,2023,2024]
regular_season_weeks = 10

for year in years:
    print(f"Scraping {year} season")
    year_game_count = 0
    
    for week in range(1,regular_season_weeks + 1):
        games = scrape_week(year,week)
        all_games.extend(games)
        year_game_count += len(games)
        
        print(f" Week {week}: {len(games)} games", end='\r')
        time.sleep(0.5)
    
    print(f"{year} complete: {len([g for g in all_games if g['year']==year])} games")

historical_df = pd.DataFrame(all_games)
print(f"\nTotal dataset: {len(historical_df)} games across {len(years)} seasons")

print("\nGames by year:")
print(historical_df['year'].value_counts().sort_index())

historical_df.to_csv('d3_football_2022_2024.csv', index=False)
print("Saved to 'd3_football_2022_2024.csv'")

Scraping 2022 season
2022 complete: 1087 games
Scraping 2023 season
2023 complete: 1083 games
Scraping 2024 season
2024 complete: 1087 games

Total dataset: 3257 games across 3 seasons

Games by year:
year
2022    1087
2023    1083
2024    1087
Name: count, dtype: int64
Saved to 'd3_football_2022_2024.csv'


In [12]:
import pandas as pd
df = pd.read_csv('d3_football_2022_2024.csv')

# Check if we have box score links
print("Columns in our data:", df.columns.tolist())

# If you don't have box score links yet, let's grab one game manually
# Go to https://www.d3football.com/scoreboard/2024/composite?view=1
# Click on any "BX" link and copy the URL

# For now, let's use an example URL (you'll need to get a real one)
test_box_score_url = "https://www.d3football.com/seasons/2022/boxscores/20220903_pwzg.xml"  # Replace with actual URL

Columns in our data: ['year', 'week', 'date', 'team1', 'score1', 'team2', 'score2']


In [16]:
import requests
from bs4 import BeautifulSoup


# You'll need to update this with an actual box score path
base_url = "https://www.d3football.com"
box_score_path = "/seasons/2022/boxscores/20220903_pwzg.xml"  # Get this from clicking a BX link

# Make the request
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}

response = requests.get(base_url + box_score_path, headers=headers)
print(f"Status: {response.status_code}")

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Save the HTML to examine it
with open('sample_box_score.html', 'w') as f:
    f.write(response.text)
print("Saved box score HTML for inspection")


game_stats = parse_complete_box_score(soup, box_score_path)

# Display results
print(f"Teams: {game_stats.get('team1', 'Unknown')} vs {game_stats.get('team2', 'Unknown')}")
print(f"\nTotal stats found: {len([k for k in game_stats.keys() if 'team1_' in k])}")

# Show all the stats we extracted
print("\nExtracted Statistics:")
print("-" * 50)

# Get all stat names (remove team1_/team2_ prefix)
stat_names = set()
for key in game_stats.keys():
    if key.startswith('team1_'):
        stat_names.add(key.replace('team1_', ''))

# Display in a nice format
for stat in sorted(stat_names):
    team1_val = game_stats.get(f'team1_{stat}', 'N/A')
    team2_val = game_stats.get(f'team2_{stat}', 'N/A')
    print(f"{stat:30} {team1_val:>10} vs {team2_val:>10}")

# Check for any errors
if game_stats['parsing_errors']:
    print("\nErrors encountered:", game_stats['parsing_errors'])        

ModuleNotFoundError: No module named 'box_scraping'